In [1]:
import tensorrt

In [2]:
from pathlib import Path
import os
venv_path = Path("../.venv")

get_libs = lambda pattern: set(venv_path.glob(pattern)) # noqa
cuda_libs = get_libs('**/libcu*.so.*') | get_libs('**/libnv*.so.*')

# path = os.environ['PATH']
# ld_library_path = os.environ['LD_LIBRARY_PATH']
# os.environ['PATH'] = f"{':'.join(str(c.resolve()) for c in cuda_libs)}:{path}"
os.environ['LD_LIBRARY_PATH'] = f"{':'.join(str(c.parent.resolve()) for c in cuda_libs)}"
print(os.environ['LD_LIBRARY_PATH'])
for l in set([c.parent.resolve() for c in cuda_libs]):
    print(l)

/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/nvidia/nvjitlink/lib:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/tensorrt_libs:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/tensorrt_libs:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/nvidia/cudnn/lib:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/nvidia/cuda_nvrtc/lib:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/nvidia/cudnn/lib:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/tensorrt_libs:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/nvidia/cuda_runtime/lib:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/nvidia/cudnn/lib:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/nvidia/cublas/lib:/home/chris/repos/deepweeds-kerascv/.venv/lib/python3.11/site-packages/nvidia/cudnn/lib:/home/chris/repos/deepwe

^ This does not actually fix the dll load issues, instead the paths need to be added to an `/etc/ld.so.conf.d/` config + added to the linker path with `ldconfig` (see dockerfile)

In [3]:
import onnxruntime as ort
model_binary = Path('../models/MeNet.onnx')

In [4]:
providers = ["TensorrtExecutionProvider", "CUDAExecutionProvider"]
onnx_session = ort.InferenceSession(model_binary, providers=providers)

In [5]:
import ctypes
ctypes.CDLL('libcudnn.so.9')

<CDLL 'libcudnn.so.9', handle 5557d04cf050 at 0x7f5f65e788d0>

In [19]:
from PIL import Image
import numpy as np

input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name

image = Image.open('../data/images/train/chinee_apple/20160928-140314-0.jpg')
image = image.convert("RGB")
batch = np.expand_dims(np.array(image).astype(np.float32), axis=0)
batch.shape

(1, 256, 256, 3)

In [28]:
results = onnx_session.run([output_name], {input_name: batch})
predictions = results[0][0]

In [29]:
classes = [
        "rubber_vine",
        "negative",
        "parthenium",
        "chinee_apple",
        "prickly_acacia",
        "snake_weed",
        "parkinsonia",
        "siam_weed",
        "lantana",
    ]

In [30]:
top_results = [
            {"class": classes[idx], "confidence": float(predictions[idx])}
            for idx in np.argsort(predictions)[::-1]
        ]

top_results

[{'class': 'chinee_apple', 'confidence': 0.810660183429718},
 {'class': 'parthenium', 'confidence': 0.07430804520845413},
 {'class': 'snake_weed', 'confidence': 0.0398639515042305},
 {'class': 'negative', 'confidence': 0.03814992308616638},
 {'class': 'lantana', 'confidence': 0.020679837092757225},
 {'class': 'rubber_vine', 'confidence': 0.011621222831308842},
 {'class': 'prickly_acacia', 'confidence': 0.003306824481114745},
 {'class': 'parkinsonia', 'confidence': 0.000989916268736124},
 {'class': 'siam_weed', 'confidence': 0.00042002645204775035}]